In [1]:
import os
import shutil

from pathlib import Path

import numpy as np
import yaml

In [2]:
os.chdir(Path().resolve().parent)

In [3]:
sessions = [
    "bex_20230621_spikes_sorted_SES",  # before
    "bex_20230624_spikes_sorted_SES",  # before
    "bex_20230629_spikes_sorted_SES",  # before
    "bex_20230630_spikes_sorted_SES",  # before
    "bex_20230701_spikes_sorted_SES",  # before
    "bex_20230708_spikes_sorted_SES",  # while
    "ken_20230614_spikes_sorted_SES",  # while and before
    "ken_20230618_spikes_sorted_SES",  # before
    "ken_20230622_spikes_sorted_SES",  # while, before and free
    "ken_20230629_spikes_sorted_SES",  # while, before and free
    "ken_20230630_spikes_sorted_SES",  # while
    "ken_20230701_spikes_sorted_SES",  # before
    "ken_20230703_spikes_sorted_SES",  # while
]

# sessions = [
#     "bex_20230221",
#     "bex_20230222",
#     "bex_20230223",
#     "bex_20230224",
#     "bex_20230225",
#     "bex_20230226",
#     "jon_20230125",
#     "jon_20230126",
#     "jon_20230127",
#     "jon_20230130",
#     "jon_20230131",
#     "jon_20230202",
#     "jon_20230203",
#     "luk_20230126",
#     "luk_20230127",
#     "luk_20230130",
#     "luk_20230131",
#     "luk_20230202",
#     "luk_20230203",
# ]

# sessions = ["ken_20230614"]
experiment = "gbyk"

In [4]:
for session in sessions:
    RESP_DIR = Path(f"data/{experiment}/{session}/responses")
    with open(RESP_DIR / "meta.yaml", "r") as f:
        meta = yaml.safe_load(f)
        resp = np.memmap(
            RESP_DIR / "data.mem",
            dtype=meta["dtype"],
            mode="r",
            shape=(meta["n_timestamps"], meta["n_signals"]),
        )
        new_meta = {k: v for k, v in meta.items()}

    duration = len(resp) - (len(resp) % 50)
    spike_count = resp[:duration].reshape(len(resp) // 50, 50, -1).sum(axis=1)

    new_meta["dtype"] = "float32"
    new_meta["sampling_rate"] = 20
    new_meta["n_timestamps"] = spike_count.shape[0]
    new_meta["n_signals"] = spike_count.shape[1]

    OUT_DIR = Path(f"data/gbyk/{session}/spike_count")
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    spike_count_memmap = np.memmap(
        os.path.join(OUT_DIR, "data.mem"),
        dtype="float32",
        mode="w+",
        shape=spike_count.shape,
    )
    spike_count_memmap[:] = spike_count[:]
    spike_count_memmap.flush()
    with open(OUT_DIR / "meta.yaml", "w") as f:
        yaml.dump(new_meta, f)

    shutil.copytree(
        RESP_DIR / "meta", Path(OUT_DIR / "meta"), dirs_exist_ok=True
    )